In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
taxi_db = create_engine("sqlite:///taxi_db.db")

In [4]:
%%time
chunk_size = 100000
x = 0
y = 1
for data_f in pd.read_csv('taxi-sample.csv', chunksize=chunk_size, iterator=True):
      data_f = data_f.rename(columns={c: c.replace(' ', '') for c in data_f.columns}) 
      data_f.index += y
      x+=1
      
      #removing duplicates
      data_f.drop_duplicates(keep='first', inplace=True)
    
      #VendorID=4 
      data_f = data_f.drop(data_f[data_f.VendorID==4].index)
    
      #removing Future Trips
        #year + time from tpep_pickup  as a series 
      col = data_f['tpep_pickup_datetime'].str.split('/')
      
      for items in col.iteritems():
        #index
        ind = items[0]
        #year  
        yr_int= int(items[1][2].split(' ')[0])
        if yr_int > 2019:
            data_f=data_f.drop(ind)
        if yr_int == 2019:
            #month
            month_int = int(items[1][0])
            if month_int > 9:
                data_f = data_f.drop(ind)
                
      # removing RateCodeID =99  
      data_f = data_f.drop(data_f[data_f.RatecodeID==99].index)
      
      #removing PULocationID =264, 265
      data_f = data_f.drop(data_f[(data_f.PULocationID==264) | (data_f.PULocationID==265)].index)
        
      #removing DOLocationID =264, 265
      data_f = data_f.drop(data_f[(data_f.DOLocationID==264) | (data_f.DOLocationID==265)].index)
    
      #removing -ve values of fare amount
      data_f = data_f.drop(data_f[data_f.fare_amount<0].index)
      
      #removing  mta_tax unknown values (other than 0,0.5)  
      data_f = data_f.drop(data_f[~((data_f.mta_tax==0) | (data_f.mta_tax==0.5))].index)
      
      #removing Extra(other than 0,0.5,1)
      data_f = data_f.drop(data_f[~((data_f.extra==0) | (data_f.extra==0.5) | (data_f.extra==1))].index)
    
    
      data_f.to_sql('taxi_Details', taxi_db, if_exists='append')
      y = data_f.index[-1] + 1
print("loading to sql_DB")

loading to sql_DB
Wall time: 29.4 s
